In [1]:
import sys
sys.path.insert(1, '/Users/linusrandud/Documents/UoM/ERP/MscDissertation/Deep-Opt')

In [2]:
import torch
import numpy as np
import json

from COProblems.MKP import MKP
from COProblems.QUBO import QUBO
from Models.DOAE import DOAE
from OptimAE import OptimAEHandler

/Users/linusrandud/anaconda3/envs/myenv_3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Highly recommended to keep as cpu for problems of size <= 100
device="cpu"
print(device)
device = torch.device(device)

cpu


## load json

In [4]:
def load_jobs_and_get_durations(json_file):
    """
    Load jobs from a JSON file and extract the durations.

    Args:
        json_file (str): Path to the JSON file containing job details.

    Returns:
        list: List of job durations.
    """
    with open(json_file, 'r') as f:
        jobs = json.load(f)
    
    durations = [job['duration'] for job in jobs.values()]
    return durations

In [5]:
filename = 'ssjsp_4'
durations = load_jobs_and_get_durations(f'../data/ssjsp/{filename}.json')
durations

[19, 1, 18, 2]

## problem converter

In [6]:
def generate_qubo_matrix(S):
    n = len(S)
    c = sum(S)
    Q = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            if i == j:
                Q[i, j] = S[i] * (S[i] - c)
            else:
                Q[i, j] = S[i] * S[j]
    
    return Q

In [7]:
Q = generate_qubo_matrix(durations)

In [8]:
Q

array([[-399.,   19.,  342.,   38.],
       [  19.,  -39.,   18.,    2.],
       [ 342.,   18., -396.,   36.],
       [  38.,    2.,   36.,  -76.]])

## save problem

In [27]:
# Generate the text file content
lines = []
lines.append("1\n")  # Number of problem instances

# Collect non-zero entries
n = Q.shape[0]
non_zero_entries = []
for i in range(n):
    for j in range(i, n):
        if Q[i][j] != 0:
            non_zero_entries.append((i + 1, j + 1, int(Q[i][j])))

# Add the number of variables and number of non-zero entries
lines.append(f"{n} {len(non_zero_entries)}\n")

# Add the non-zero entries to lines
for entry in non_zero_entries:
    i, j, value = entry
    lines.append(f"{i} {j} {value}\n")

# Write to file
with open(f"../data/qubo/{filename}.txt", "w") as f:
    f.writelines(lines)